<a href="https://colab.research.google.com/github/kgarvita/MISC/blob/main/Local_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In AWS instance : foldername/localGPT/privategpt/ingest_local_model


In [ ]:
#install packages
!pip install sentence_transformers
!pip install langchain
!pip install chroma

In [ ]:
#Save model in model folder "ggml-gpt4all-j-v1.3-groovy.bin"
#cell 2 - download sentence_transformer model in file embedding_load.py.
#model then is saved in folder "embedding_model" in the project folder.


In [ ]:
!pip install sentence_transformers

In [ ]:
#embedding_load.py
from sentence_transformers import SentenceTransformer
#emb_model = SentenceTransformer('all-MiniLM-L6-v2')
emb_model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
emb_model.save('sentance_transformer')


In [ ]:
#ingest_local_model.py
#Creating embedding using embedding model and perform similarity seach of the database
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
#Using locally stored embedding model to create embeddings for the text
#model = SentenceTransformer('/workdir/privateGPT/localgpt/privateGPT/embedding_model',device='cpu')
model = SentenceTransformer('/workdir/privateGPT/localgpt/privateGPT/sentance_transformer',device='cpu')
print(model)

embedded_vector = model.encode("What is your name?")
print(f"Size of embedded vector: {len(embedded_vector)}")

#Load and break the text document
from langchain.document_loaders import TextLoader

# load the document and split it into chunks
#loader = TextLoader("source_documents/state_of_the_union.txt")
#documents = loader.load()

#load pdf document
from langchain.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("source_documents/Policy_Travel.pdf")
documents = loader.load()
# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
#print(docs)

# create the open-source embedding function
#embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2",cache_folder='embedding_model')
embedding_function = SentenceTransformerEmbeddings(model_name="multi-qa-mpnet-base-dot-v1",cache_folder='sentance_transformer')
# save the database on the disk
db = Chroma.from_documents(docs, embedding_function, persist_directory="./chroma_db")
db.persist()

## Test the semantic retrieval
print(db.similarity_search(query="Who can authorize the travel?", k= 4))


In [1]:
#query.py
#setting the dabase in retrieval mode and perform search
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from chromadb.config import Settings
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
PERSIST_DIRECTORY="chroma_db"
EMBEDDINGS_MODEL_NAME = "all-MiniLM-L6-v2"

# Define the Chroma settings
CHROMA_SETTINGS = Settings(
        persist_directory=PERSIST_DIRECTORY,
        anonymized_telemetry=False
)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2",cache_folder='embedding_model')
db = Chroma(persist_directory=PERSIST_DIRECTORY, embedding_function=embedding_function)
retriever = db.as_retriever()

#Test the retriever
#print(retriever.vectorstore.similarity_search(query = "What happens if flight gets canceled?",k=2))

#Load gpt4all model
from langchain.llms import GPT4All

MODEL_N_BATCH=8
TARGET_SOURCE_CHUNKS=4
MODEL_PATH = "models/ggml-gpt4all-j-v1.3-groovy.bin"
MODEL_N_CTX=1000
# Prepare the LLM
#llm = GPT4All(model=MODEL_PATH, n_ctx=MODEL_N_CTX, backend='gptj', callbacks=None, verbose=False)
llm = GPT4All(model=MODEL_PATH, max_tokens=2000, backend='gptj', n_batch=8, callbacks=None, verbose=True)

#Initialize question answer chain
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=True)
query = "What happens if flight gets canceled?"
res = qa(query)
answer, docs = res['result'], res['source_documents']

# Get the answer from the chain
# Print the result
print("\n\n> Question:")
print(query)
print("\n> Answer:")


ModuleNotFoundError: ignored